#### Enviroment
Python 3.11.0

-----------------------------

### Libraries

In [2]:
from tabula.io import read_pdf
import tabula
import os

#### This chunk of the code is for you to know how to read PDF files but they are not going to be availabe becase contains sensible informatios of my number accounts.
What matters here is the kind of expenses, where they were and how much money.

In [5]:
dir = os.getcwd()
dir_path = dir+'\Extractos'   # Just to find the path where de bank resume of expenses are available

lista_pdf = []  # List to store de name of .pdf

for files in os.listdir(dir_path):
    if files.endswith('pdf'):  #just files *.pdf
        lista_pdf.append(files)
    
print(lista_pdf)

['Banco en Línea - Santander1.pdf', 'Banco en Línea - Santander2.pdf', 'Banco en Línea - Santander3.pdf', 'Banco en Línea - Santander4.pdf']


### Transform data to CSV using the librarie Tabula 

In [6]:
dir = os.getcwd()
dir_path = dir+'\Extractos\\'  

for df in lista_pdf:
   tabula.convert_into(dir_path + df, "output"+df+".csv", output_format="csv", pages='all')

#### Conver all files to CSV format

In [7]:
dir_path = os.getcwd()

lista_csv = []

for files in os.listdir(dir_path):
    if files.endswith('csv'):
        lista_csv.append(files)
    
print(lista_csv)

['outputBanco en Línea - Santander1.pdf.csv', 'outputBanco en Línea - Santander2.pdf.csv', 'outputBanco en Línea - Santander3.pdf.csv', 'outputBanco en Línea - Santander4.pdf.csv']


### Cleancy 
in this part we are going to prepare de files to be correctly read
it involves to erase some rows because the pdf files have certain lines of information at the begining and end that is not necesary and might make the data dirty

In [9]:
import pandas as pd
print(pd.options.display.max_rows) 

colnames = ['Fecha', 'Salida', 'Entrada',  'Descripción','Documento']

df_s0 =  pd.read_csv(lista_csv[0], encoding='Latin1',skiprows=[0,1], skipfooter=4, engine='python',names=colnames)
df_s1 =  pd.read_csv(lista_csv[1], encoding='Latin1',skiprows=[0,1], skipfooter=4,engine='python',  names=colnames)
df_s2 =  pd.read_csv(lista_csv[2], encoding='Latin1',skiprows=[0,1],skipfooter=4, engine='python', names=colnames)
df_s3 =  pd.read_csv(lista_csv[3], encoding='Latin1',skiprows=[0,1], skipfooter=4, engine='python', names=colnames)

60


#### Next process is to cancatenate all the files so we can work just with one , operartions are easy that way in this case

In [12]:
df_s_all = pd.concat([df_s0,df_s1,df_s2,df_s3])
df_s_all.head()

,Fecha,Salida,Entrada,Descripción,Documento
0,03/01/2022,763.400,NaN,Transf. Internet a otro Bancos,0
1,03/01/2022,189.390,NaN,Traspaso Internet a T. Crédito,0
2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0
3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0
4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0


#### From this point on we are going to work just with purchase because de propose of this analisys has to be with how much I am spending and in what

In [33]:
df_purchase = df_s_all[df_s_all['Descripción'].str.contains('Compra') ]

In [34]:
df_purchase.to_csv('Purchase.csv')


#### df_bnk_purchases is the dataset to work, it doesn't contain sensible data so it will be available in this repository of Github

In [35]:
df_bnk_purchases= pd.read_csv("Purchase.csv")

In [36]:
# Lets check the amount of NA in each of the columns of the dataset
df_bnk_purchases.isnull().sum()

Unnamed: 0       0
Fecha            0
Salida           0
Entrada        487
Descripción      0
Documento        0
dtype: int64

In [37]:
df_bnk_purchases.head()

,Unnamed: 0,Fecha,Salida,Entrada,Descripción,Documento
0,2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0
1,3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0
2,4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0
3,5,03/01/2022,8.350,NaN,Compra STARBUCKS ESTADO,0
4,6,03/01/2022,3.000,NaN,Compra POLLA CHILENA,0


##### As we can see there is NA data in the column Entrada this is normal because the movments we are analysing are outcomes, in fact we can drop this column since it is not necesary

#### On the other hand the column Descripcion has importan information that can help uo to segment information later for example:
- Transferences
- Who I transfered money
- Purchases
- Places where I purchase

In [39]:
purchases_des = df_bnk_purchases["Descripción"].str.split(' ', 2  ,expand=True).reset_index(drop=True)
purchases_des.head(10)

C:\Users\yohan\AppData\Local\Temp\ipykernel_20016\1047782853.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  purchases_des = df_bnk_purchases["Descripción"].str.split(' ', 2  ,expand=True).reset_index(drop=True)


,0,1,2
0,Compra,TOTTUS,VICUNA MAC
1,Compra,BEST,FOR PETS
2,Compra,SUPERMERCADO,LOS
3,Compra,STARBUCKS,ESTADO
4,Compra,POLLA,CHILENA
5,Compra,NOTARIA,ELENA TOR
6,Compra,PEPE,None
7,Compra,NOTARIA,LASCAR
8,Compra,SN,CAMILO METRO I
9,Compra,SODIMAC,PARQUE AR


#### Most of the information in the Descripción column has three states 0= type, 1 =Category and 3 = place
Lets replace the names of the columns

In [42]:
purchases_ren = purchases_des.rename(columns= {0:'Tipo',1:'Categoria',2:'Lugar'})
purchases_ren.head(10)

,Tipo,Categoria,Lugar
0,Compra,TOTTUS,VICUNA MAC
1,Compra,BEST,FOR PETS
2,Compra,SUPERMERCADO,LOS
3,Compra,STARBUCKS,ESTADO
4,Compra,POLLA,CHILENA
5,Compra,NOTARIA,ELENA TOR
6,Compra,PEPE,None
7,Compra,NOTARIA,LASCAR
8,Compra,SN,CAMILO METRO I
9,Compra,SODIMAC,PARQUE AR


In [43]:
df_purchases_join = df_bnk_purchases.join(purchases_ren).reset_index(drop=True)
df_purchases_join.head()

,Unnamed: 0,Fecha,Salida,Entrada,Descripción,Documento,Tipo,Categoria,Lugar
0,2,03/01/2022,30.092,NaN,Compra TOTTUS VICUNA MAC,0,Compra,TOTTUS,VICUNA MAC
1,3,03/01/2022,12.210,NaN,Compra BEST FOR PETS,0,Compra,BEST,FOR PETS
2,4,03/01/2022,9.360,NaN,Compra SUPERMERCADO LOS,0,Compra,SUPERMERCADO,LOS
3,5,03/01/2022,8.350,NaN,Compra STARBUCKS ESTADO,0,Compra,STARBUCKS,ESTADO
4,6,03/01/2022,3.000,NaN,Compra POLLA CHILENA,0,Compra,POLLA,CHILENA


In [44]:
# Let see the shape of the Dataset
df_purchases_join.shape

(487, 9)

#### Anothe split that can be usefull is Date, we have to separate year, month and day in order to create a granular analisys and see if there is a patern in the way I spend money

In [45]:
# To see if I can split dates I have to convert Fecha into a date type, for now is an object
df_purchases_join.dtypes

Unnamed: 0       int64
Fecha           object
Salida         float64
Entrada        float64
Descripción     object
Documento        int64
Tipo            object
Categoria       object
Lugar           object
dtype: object